In [ ]:
from matplotlib import pyplot as plt
from matplotlib import patches
import rasterio
from rasterio.plot import show as rastershow

import numpy as np
import pandas as pd
import os
import glob

import geopandas as gpd

from shapely.geometry import Point, Polygon

## Quick check for processed data

After running the preprocessing script, run this notebook for a quick sanity check of the processed outputs.

All data is located in `/work/ka1176/shared_data/2024-ufz-deeptree/polygon-labelling/`.

These are all the labeled tiles:

In [ ]:
shapes = np.sort(glob.glob('/work/ka1176/shared_data/2024-ufz-deeptree/polygon-labelling/labels/label_tile_*.shp'))

In [ ]:
shapes

In [ ]:
print(f'Number of available shape files for tiles: {len(shapes)}')

We process one of the label files and check the coordinate reference system (CRS). The label files need to be cast to the same CRS as the raster images. They are given in `EPSG:4326` and are now cast to `EPSG:25832`.

In [ ]:
polygons = gpd.read_file('/work/ka1176/shared_data/2024-ufz-deeptree/polygon-labelling/labels/label_tile_1_1.shp')
polygons.crs

In [ ]:
polygons

In [ ]:
rastertif = rasterio.open(os.path.join('/work/ka1176/shared_data/2024-ufz-deeptree/polygon-labelling/', 'tiles', 'tile_0_1.tif'))
rastertif.crs

In [ ]:
for shape in shapes:
    print(gpd.read_file(shape).crs)

In [ ]:
polygons = polygons.set_crs(epsg=4326) # needed this explicitly
polygons = polygons.to_crs(epsg=25832)
polygons.crs

For preprocessing and plots, we need all polygons combined in one file. This is created here and saved in `/work/ka1176/shared_data/2024-ufz-deeptree/polygon-labelling/labels/all_labels.shp`.

In [ ]:
def fix_crs(shape, is_crs=4326, target_crs=25832):
    '''Fix the CRS if necessary'''
    if shape.crs is None: # naive coords
        shape.crs = is_crs

    return shape.to_crs(epsg=target_crs)

In [ ]:
all_polygons = pd.concat([fix_crs(gpd.read_file(shape)).assign(tile=shape) for shape in shapes])
#all_polygons.drop(columns='tile').to_file('/work/ka1176/shared_data/2024-ufz-deeptree/polygon-labelling/labels/all_labels_class0-2.shp')

In [ ]:
#all_polygons = all_polygons[all_polygons['class'] < 2]
#all_polygons.drop(columns='tile').to_file('/work/ka1176/shared_data/2024-ufz-deeptree/polygon-labelling/labels/all_labels.shp')

In [ ]:
all_polygons.head()

In [ ]:
all_polygons.info()

### Check the preprocessed raster files

We are starting with 
- tiles
- labels

In preprocessing, we created
- masks
- outlines
- distance transforms

In [ ]:
tiles = [shape.replace('labels/', 'tiles/').replace('label_', '').replace('.shp', '.tif') for shape in shapes]
masks = [shape.replace('labels/', 'masks/').replace('label_tile_', 'mask_').replace('.shp', '.tif') for shape in shapes]
outlines = [shape.replace('labels/', 'outlines/').replace('label_tile_', 'outline_').replace('.shp', '.tif') for shape in shapes]
distance_transforms = [shape.replace('labels/', 'dist_trafo/').replace('label_tile_', 'dist_trafo_').replace('.shp', '.tif') for shape in shapes]

In [ ]:
all_rastertifs = [rasterio.open(tile) for tile in tiles]
all_bounds = [rastertif.bounds for rastertif in all_rastertifs]
all_extents = [(b.left, b.bottom, b.right, b.top) for b in all_bounds]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
# add original tiles
for rastertif in all_rastertifs:
    rastershow(rastertif, ax=ax)

# add bounding boxes to display
for extent in all_extents:
    rc = patches.Rectangle((extent[0], extent[1]), extent[2]-extent[0], extent[3]-extent[1],
                           lw=2, fc='none', ec='lightgray',
                          )
    ax.add_patch(rc)

#### Sanity check for masks

The following plot shows all the labeled tiles (RGB images) and the labeled polygons (cyan). The masks are overlayed in blue.

The cyan outlines should match the masks.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
# add original tiles
for rastertif in all_rastertifs:
    rastershow(rastertif, ax=ax)
    break

# all labeled polygons
all_polygons.plot(column='class', ax=ax, facecolor='none', cmap='jet', legend=True)

# check if mask is correct
for mask in masks:
    rastershow(rasterio.open(mask), ax=ax, alpha=0.35, cmap='Blues_r', with_bounds=True)
    break

# add bounding boxes to display
for extent in all_extents:
    rc = patches.Rectangle((extent[0], extent[1]), extent[2]-extent[0], extent[3]-extent[1],
                           lw=2, fc='none', ec='lightgray',
                          )
    ax.add_patch(rc)

ax.legend()

In [ ]:
len(all_rastertifs)

In [ ]:
fig, axs = plt.subplots(3, 4, figsize=(20, 12))
axs = axs.flatten()

# add original tiles
for rastertif, mask, ax in zip(all_rastertifs, masks, axs):
    rastershow(rastertif, ax=ax)

    # all labeled polygons
    all_polygons.plot(column='class', ax=ax, facecolor='none', cmap='jet', legend=True)

    # check if mask is correct
    rastershow(rasterio.open(mask), ax=ax, alpha=0.35, cmap='Reds_r', with_bounds=True)


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
# add original tiles
for rastertif in all_rastertifs:
    rastershow(rastertif, ax=ax)

# check if mask is correct
for mask in masks:
    rastershow(rasterio.open(mask), ax=ax, alpha=0.35, cmap='Blues_r', with_bounds=True)

# add bounding boxes to display
for extent in all_extents:
    rc = patches.Rectangle((extent[0], extent[1]), extent[2]-extent[0], extent[3]-extent[1],
                           lw=2, fc='none', ec='lightgray',
                          )
    ax.add_patch(rc)

# all labeled polygons
all_polygons.plot(column='tile', ax=ax, facecolor='none', edgecolor='cyan')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

# all labeled polygons
all_polygons.plot(column='class', ax=ax, facecolor='none', edgecolor='cyan')

#### Sanity check for outlines

The outlines are a bit hard to see in the overlay, so we perform the sanity check by comparing them against the masks. The outlines (red) should match the polygons (blue).

In [ ]:
fig, ax = plt.subplots(1, 1, sharex=True, sharey=True)

k = 7 # tile index

mask = rasterio.open(masks[k]).read()
ax.imshow(mask.squeeze(), cmap='Blues')



all_polygons.plot(column='class', ax=ax, facecolor='none', cmap='jet', legend=True)

outline = rasterio.open(outlines[k]).read()
ax.imshow(outline.squeeze(), cmap='Reds', alpha=0.5)

fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, sharex=True, sharey=True)

k = 7 # tile index





all_polygons.plot(column='class', ax=ax, facecolor='none', cmap='jet', legend=True)
rastershow(rasterio.open(masks[k]), ax=ax)

#outline = rasterio.open(outlines[k]).read()
#ax.imshow(outline.squeeze(), cmap='Reds', alpha=0.5)

fig.tight_layout()
plt.show()

#### Sanity check for distance transforms

The distance transforms denotes the distance of a point within the polygon to its boundary. Again, the shapes should match. The heatmap colors inside the polygons should reflect the distance to the boundary.

In [ ]:
fig, ax = plt.subplots(1, 1, sharex=True, sharey=True)

mask = rasterio.open(masks[k]).read()
ax.imshow(mask.squeeze(), cmap='Blues')

dist_trafo = rasterio.open(distance_transforms[k]).read()

ax.imshow(dist_trafo.squeeze(), cmap='magma', alpha=0.5)

fig.tight_layout()
plt.show()

In [ ]:
import numpy as np

In [ ]:
rng1 = np.random.default_rng(100)

In [ ]:
rng2 = np.random.default_rng(200)

In [ ]:
for i in range(10):
    print(rng1.random())

In [ ]:
import time

In [ ]:
int(time.time())

In [ ]:
import rioxarray

In [ ]:
tile = rioxarray.op

In [ ]:
.isel(x=slice(None, None, -1))

In [1]:
import os

In [3]:
os.listdir('/work/ka1176/harsh/Tree_Delineation/TreeCrownDelineation_DKRZ/results/inference_halle_active_learning/Entropy maps visualized/save_prediction_dir')

['tile_13_6_pred.tif',
 'tile_5_16.png',
 'tile_1_0.png',
 'tile_6_0_pred.tif',
 'tile_16_4.png',
 'tile_13_6.png',
 'tile_9_1.png',
 'tile_8_11_pred.tif',
 'tile_9_8_pred.tif',
 'tile_0_16.png',
 'tile_17_13.png',
 'tile_13_1_pred.tif',
 'tile_12_2_pred.tif',
 'tile_10_9.png',
 'tile_18_16_pred.tif',
 'tile_15_12.png',
 'tile_13_14_pred.tif',
 'tile_8_18_pred.tif',
 'tile_13_5_pred.tif',
 'tile_2_16_pred.tif',
 'tile_11_18_pred.tif',
 'tile_7_0_pred.tif',
 'tile_17_10_pred.tif',
 'tile_13_2_pred.tif',
 'tile_1_13_pred.tif',
 'tile_16_3_pred.tif',
 'tile_11_12_pred.tif',
 'tile_1_5_pred.tif',
 'tile_14_16.png',
 'tile_8_14_pred.tif',
 'tile_16_14_pred.tif',
 'tile_5_17_pred.tif',
 'tile_1_16.png',
 'tile_8_19.png',
 'tile_0_2_pred.tif',
 'tile_17_17_pred.tif',
 'tile_7_13.png',
 'tile_10_1.png',
 'tile_15_5_pred.tif',
 'tile_19_9_pred.tif',
 'tile_9_15_pred.tif',
 'tile_4_12_pred.tif',
 'tile_15_14_pred.tif',
 'tile_12_13.png',
 'tile_15_2.png',
 'tile_17_3.png',
 'tile_13_8.png',
 'ti